**Chatbot using deep learning**

Tflearn is used instead of keras. Tflearn is also a deep learning library which is built on top of tensorflow. With Tflearn we can use python arrays ddirectly while keras needs numpy arrays. Tflearn provides better performance than keras. Only disadvantage is the lack of easily integrated pre-trained models.

We will train our model on basis of our intent file intent.json
Intent file will contain tags, patterns and responses.

Tags: Unique name

Patterns: a sentence pattern for neural network classifier

Responses:  Responses to pattern

nltk: Natural language tool kit

'punkt': tokenize sentences into individual words



In [1]:
# Libraries for NLP
import nltk
nltk.download('punkt')
from nltk.stem.lancaster import LancasterStemmer
stemmer = LancasterStemmer()

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [6]:
#Libraries for Tensorflow
import tensorflow as tf
import numpy as np
import tflearn
import random
import json

In [8]:
from google.colab import files
files.upload()

Saving intents.json to intents.json


{'intents.json': b'{"intents": [\r\n        {"tag": "greeting",\r\n         "patterns": ["Hi", "How are you", "Is anyone there?", "Hello", "Good day"],\r\n         "responses": ["Hello, thanks for visiting", "Good to see you again", "Hi there, how can I help?"],\r\n         "context_set": ""\r\n        },\r\n        {"tag": "goodbye",\r\n         "patterns": ["Bye", "See you later", "Goodbye"],\r\n         "responses": ["See you later, thanks for visiting", "Have a nice day", "Bye! Come back again soon."]\r\n        },\r\n        {"tag": "thanks",\r\n         "patterns": ["Thanks", "Thank you", "That\'s helpful"],\r\n         "responses": ["Happy to help!", "Any time!", "My pleasure"]\r\n        },\r\n        {"tag": "hours",\r\n         "patterns": ["What hours are you open?", "What are your hours?", "When are you open?" ],\r\n         "responses": ["We\'re open every day 9am-9pm", "Our hours are 9am-9pm every day"]\r\n        },\r\n        {"tag": "location",\r\n         "patterns": 

In [9]:
with open('intents.json') as json_file:
  intents = json.load(json_file)

intents

{'intents': [{'context_set': '',
   'patterns': ['Hi', 'How are you', 'Is anyone there?', 'Hello', 'Good day'],
   'responses': ['Hello, thanks for visiting',
    'Good to see you again',
    'Hi there, how can I help?'],
   'tag': 'greeting'},
  {'patterns': ['Bye', 'See you later', 'Goodbye'],
   'responses': ['See you later, thanks for visiting',
    'Have a nice day',
    'Bye! Come back again soon.'],
   'tag': 'goodbye'},
  {'patterns': ['Thanks', 'Thank you', "That's helpful"],
   'responses': ['Happy to help!', 'Any time!', 'My pleasure'],
   'tag': 'thanks'},
  {'patterns': ['What hours are you open?',
    'What are your hours?',
    'When are you open?'],
   'responses': ["We're open every day 9am-9pm",
    'Our hours are 9am-9pm every day'],
   'tag': 'hours'},
  {'patterns': ['What is your location?',
    'Where are you located?',
    'What is your address?',
    'Where is your restaurant situated?'],
   'responses': ['We are on the intersection of London Alley and Bridge A

In [11]:
words = []
tags = []
documents = []
ignore = ['?']

for intent in intents['intents']:
  for pattern in intent['patterns']:
    #tokenize every word in patterns
    x = nltk.word_tokenize(pattern)
    #add words to words list
    words.extend(x)
    documents.append((x,intent['tag']))
    if intent['tag'] not in tags:
      tags.append(intent['tag'])

In [15]:
#Perform Stemming
words = [stemmer.stem(x.lower()) for x in words if x not in ignore]
words = sorted(list(set(words)))

tags = sorted(list(set(tags)))

print(len(documents), 'documents')
print(len(tags),'tags',tags)
print(len(words),'words',words)

31 documents
9 tags ['deliveryoption', 'goodbye', 'greeting', 'hours', 'location', 'menu', 'payments', 'thanks', 'todaysmenu']
57 words ["'s", 'acceiv', 'address', 'anyon', 'ar', 'bye', 'can', 'card', 'cash', 'cours', 'credit', 'day', 'del', 'delicy', 'delivery', 'dish', 'do', 'food', 'for', 'from', 'good', 'goodby', 'hello', 'help', 'hi', 'hom', 'hour', 'how', 'is', 'lat', 'loc', 'main', 'mastercard', 'me', 'menu', 'most', 'on', 'op', 'opt', 'provid', 'resta', 'see', 'serv', 'situ', 'spec', 'tak', 'tel', 'thank', 'that', 'the', 'ther', 'today', 'what', 'when', 'wher', 'yo', 'you']


In [18]:
#training data
training = []
output = []
output_empty = [0]*len(tags)

for doc in documents:
  bag = []
  pattern_words = doc[0]
  pattern_words = [stemmer.stem(word.lower()) for word in pattern_words]
  for w in words:
    bag.append(1) if w in pattern_words else bag.append(0)
  
  output_row = list(output_empty)
  output_row[tags.index(doc[1])] = 1
  training.append([bag,output_row])

random.shuffle(training)
training = np.array(training)

train_x = list(training[:,0])
train_y = list(training[:,1])


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:18: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray


In [24]:
#resetting underlying graph
tf.compat.v1.reset_default_graph()

#Building Neural Net
net = tflearn.input_data(shape=[None,len(train_x[0])])
net = tflearn.fully_connected(net,10)
net = tflearn.fully_connected(net,10)
net = tflearn.fully_connected(net,len(train_y[0]),activation='softmax')
net = tflearn.regression(net)

#Define model and set up tensorboard
model = tflearn.DNN(net,tensorboard_dir='tflearn_logs')



Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


In [25]:
#Start Training
model.fit(train_x, train_y, n_epoch = 1000, batch_size = 8, show_metric=True)
model.save('chatbot.tflearn')

Training Step: 3999  | total loss: 0.01208 | time: 0.016s
| Adam | epoch: 1000 | loss: 0.01208 - acc: 1.0000 -- iter: 24/31
Training Step: 4000  | total loss: 0.01220 | time: 0.021s
| Adam | epoch: 1000 | loss: 0.01220 - acc: 1.0000 -- iter: 31/31
--
INFO:tensorflow:/content/chatbot.tflearn is not in all_model_checkpoint_paths. Manually adding it.


In [26]:
model.load('./chatbot.tflearn')

INFO:tensorflow:Restoring parameters from /content/chatbot.tflearn


In [48]:
def cleanup(sentence):
   sentence_words = nltk.word_tokenize(sentence)
   sentence_words = [stemmer.stem(word.lower()) for word in sentence_words]
   return sentence_words

def bag(sentence, words, show_details=False):
  sentence_words = cleanup(sentence)
    # generating bag of words
  bag = [0]*len(words)  
  for s in sentence_words:
    for i,w in enumerate(words):
      if w == s: 
        bag[i] = 1
        if show_details:
          print ("found in bag: %s" % w)

  return(np.array(bag))

In [61]:
ERROR_THRESHOLD = 0.3

def classify(sentence):
  #Generate probabilities from the model
  results = model.predict([bag(sentence,words)])[0]
  #Filterout predictions below threshold
  results = [[i,r] for i,r in enumerate(results) if r>ERROR_THRESHOLD]
  # sort by probability
  results.sort(key=lambda x: x[1], reverse=True)
  return_list=[]
  for r in results:
    return_list.append((tags[r[0]],r[1]))
  return return_list

def response(sentence, userID='123', show_details = False):
  results = classify(sentence)
  if results:
    while results:
      for i in intents['intents']:
        if i['tag'] == results[0][0]:
          return print(random.choice(i['responses']))
      results.pop(0)

In [55]:
classify('What are your hours of operations?')

[('hours', 0.9985233)]

In [64]:
response('What is menu for today?')

Our speciality for today is Chicken Tikka


{}